# Present Wrapping Problem

In [2]:
%load_ext autoreload
%load_ext iminizinc
%matplotlib inline
%autoreload 2

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from z3 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


## Functions

In [3]:
def read_input(path):
    file = open(path,"r").readlines()
    w_paper , h_paper = tuple(map(int, file[0].rstrip("\n").split(" ")))
    n = int(file[1].rstrip("\n"))
    presents = []
    for i in range(2, n + 2):
        presents.append(list(map(int, file[i].rstrip("\n").split(" "))))
    return w_paper, h_paper, n, presents

In [4]:
def show_solutions(w_paper, h_paper, n, presents, solutions):
    colours = np.random.rand(n, 3)
    if not isinstance(solutions, list):
        solutions = [solutions]
    for x in solutions:
        bl_corners = x['bl_corners'] if 'bl_corners' in x else x
        show_solution(w_paper, h_paper, n, presents, bl_corners, colours)

def show_solution(w_paper, h_paper, n,presents, bl_corners, colours=None):
    if colours is None:
        colours = np.random.rand(n, 3)
    fig, ax = plt.subplots()
    for i in range(n):
        ax.add_patch(patches.Rectangle(
            bl_corners[i],
            presents[i][0],
            presents[i][1],
            facecolor=colours[i]
        ))
    ax.set_xlim(0, w_paper)
    ax.set_ylim(0, h_paper)
    plt.show()
    print(f"Solution: {bl_corners}")

In [5]:
def order_presents(presents, method='height', reverse=False):
    to_sort = (
        [e[0] * e[1] for e in presents] if method == 'area'
        else [e[1] for e in presents] if method == 'height'
        else [e[0] for e in presents] if method == 'width'
        else None
    )
    sorted_indices = sorted(range(len(to_sort)), key=lambda k: to_sort[k], reverse=reverse)
    presents = [presents[i] for i in sorted_indices]
    return presents, sorted_indices

## MiniZinc model

### Input

In [32]:
instance = 28
min_instance, max_instance = 8, 40

In [34]:
if instance is not None:
    min_instance, max_instance = instance, instance
for i in range(min_instance, max_instance + 1):
    path = f"instances/{i}x{i}.txt"
    w_paper, h_paper, n, presents = read_input(path)
    unordered_presents = list(presents)
    presents, sorted_indices = order_presents(presents, method='area', reverse=True)
    bl_corners = []
    print(f"Paper dimensions: {i}x{i}")
    print(f"Presents dimensions: {presents}")
    solutions = %minizinc -s pwp-v5
    if solutions:
        show_solutions(w_paper, h_paper, n, presents, solutions)

784
Paper dimensions: 28x28
Presents dimensions: [[4, 23], [3, 23], [5, 13], [3, 21], [4, 13], [3, 17], [3, 16], [5, 9], [3, 11], [4, 8], [3, 10], [5, 6], [3, 9], [3, 8], [3, 7], [4, 5], [3, 6], [4, 4], [3, 5], [3, 4], [4, 3], [3, 3]]
Error in MiniZinc:

Col_Stacking: true


Row_Stacking: false

%%%mzn-stat: nSolutions=0



### First model 

In [88]:
%%mzn_model pwp-v1
include "globals.mzn";
int: w_paper;
int: h_paper;
int: n;
array[1..n, 1..2] of var 0..max(w_paper, h_paper) - 1: bl_corners;
array[1..n, 1..2] of int: presents;
constraint forall(i in 1..n) (bl_corners[i, 1] + presents[i, 1] <= w_paper);
constraint forall(i in 1..n) (bl_corners[i, 2] + presents[i, 2] <= h_paper);
constraint forall(i, j in 1..n where j > i) (
    max(bl_corners[i, 1], bl_corners[j, 1]) >= min(bl_corners[i, 1] + presents[i, 1], bl_corners[j, 1] + presents[j, 1]) \/
    max(bl_corners[i, 2], bl_corners[j, 2]) >= min(bl_corners[i, 2] + presents[i, 2], bl_corners[j, 2] + presents[j, 2])
);
solve satisfy;

### Second model

In [93]:
%%mzn_model pwp-v2
include "globals.mzn";
int: w_paper;
int: h_paper;
int: n;
array[1..n, 1..2] of int: presents;
int: max_dim = max(w_paper, h_paper);
int: min_present_dim = min([min(presents[i, 1], presents[i, 2]) | i in 1..n]);

function var int: coord_to_value(var int: x, var int: y) = x + y * (max_dim + 1);
function var int: x_overlap(int: i, int: j) = 
    max(0, min(bl_corners[i, 1] + presents[i, 1], bl_corners[j, 1] + presents[j, 1]) - max(bl_corners[i, 1], bl_corners[j, 1]));
function var int: y_overlap(int: i, int: j) = 
    max(0, min(bl_corners[i, 2] + presents[i, 2], bl_corners[j, 2] + presents[j, 2]) - max(bl_corners[i, 2], bl_corners[j, 2]));
function var int: overlap_area(int: i, int: j) = x_overlap(i, j) * y_overlap(i, j);

array[1..n, 1..2] of var 0..max_dim - min_present_dim: bl_corners;
array[1..n] of var 0..max_dim * max_dim: bl_corners_values;

predicate values_channeling(int: i) = 
    forall(i in 1..n) (bl_corners_values[i] == coord_to_value(bl_corners[i, 1], bl_corners[i, 2]));
constraint forall(i in 1..n) (values_channeling(i));

constraint alldifferent(bl_corners_values);
constraint count_eq(bl_corners_values, 0, 1);
constraint forall(i in 1..n) (bl_corners[i, 1] + presents[i, 1] <= w_paper);
constraint forall(i in 1..n) (bl_corners[i, 2] + presents[i, 2] <= h_paper);
constraint forall(i, j in 1..n where j > i) (overlap_area(i, j) == 0);
solve satisfy;

### Third model

In [97]:
%%mzn_model pwp-v3
include "globals.mzn";
int: w_paper;
int: h_paper;
int: n;
int: max_dim = max(w_paper, h_paper);
array[1..n, 1..2] of 1..max_dim: presents;
array[1..n] of 1..max_dim: presents_xs = [presents[i, 1] | i in 1..n];
array[1..n] of 1..max_dim: presents_ys = [presents[i, 2] | i in 1..n];
int: min_present_dim = min([min(presents[i, 1], presents[i, 2]) | i in 1..n]);

function var int: coord_to_value(var int: x, var int: y) = x + y * (max_dim + 1);

array[1..n, 1..2] of var 0..max_dim - min_present_dim: bl_corners;
array[1..n, 1..2] of var 1..max_dim: tr_corners;
array[1..n] of var 0..max_dim * max_dim: bl_corners_values;
array[1..n] of var 0..w_paper - min_present_dim: bl_corners_xs;
array[1..n] of var 0..h_paper - min_present_dim: bl_corners_ys;

predicate values_channeling(int: i) = 
    forall(i in 1..n) (bl_corners_values[i] == coord_to_value(bl_corners[i, 1], bl_corners[i, 2]));
constraint forall(i in 1..n) (values_channeling(i));

predicate bl_corners_channeling(int: i) = 
    forall(i in 1..n) (bl_corners[i, 1] == bl_corners_xs[i] /\ bl_corners[i, 2] == bl_corners_ys[i]);
constraint forall(i in 1..n) (bl_corners_channeling(i));

predicate tr_corners_channeling(int: i) = 
    forall(i in 1..n) (bl_corners[i, 1] + presents[i, 1] == tr_corners[i, 1] /\ bl_corners[i, 2] + presents[i, 2] == tr_corners[i, 2]);
constraint forall(i in 1..n) (tr_corners_channeling(i));

constraint alldifferent(bl_corners_values);
constraint count_eq(bl_corners_values, 0, 1);
constraint diffn_k(bl_corners, presents);
constraint cumulative(bl_corners_xs, presents_xs, presents_ys, h_paper);
constraint cumulative(bl_corners_ys, presents_ys, presents_xs, w_paper);
solve satisfy;

### Fourth model

In [127]:
%%mzn_model pwp-v4
include "globals.mzn";

%Input values
int: w_paper;
int: h_paper;
int: n;
int: max_dim = max(w_paper, h_paper);
int: total_paper_area = w_paper * h_paper;
int: total_presents_area = sum(areas);
int: min_present_width = min(presents_xs);
int: min_present_height = min(presents_ys);
int: max_present_width = max(presents_xs);
int: max_present_height = max(presents_ys);
int: min_present_dim = min([min_present_width, min_present_height]);
int: max_present_dim = max([max_present_width, max_present_height]);
int: max_present_ind = arg_max(areas);
set of int: ns = 1..n;
array[ns, 1..2] of int: presents;
array[ns] of int: presents_xs = [presents[i, 1] | i in ns];
array[ns] of int: presents_ys = [presents[i, 2] | i in ns];
array[ns] of int: areas = [presents_xs[i] * presents_ys[i] | i in ns];

% Variables
array[ns, 1..2] of var 0..max_dim - min_present_dim: bl_corners;
array[ns, 1..2] of var min_present_dim..max_dim: tr_corners;
array[ns] of var 0..(w_paper * h_paper) - 1: bl_corners_values;
array[ns] of var 0..w_paper - min_present_width: bl_corners_xs;
array[ns] of var 0..h_paper - min_present_height: bl_corners_ys;

% Functions
function var int: coord_to_value(var int: x, var int: y) = x + (y * max_dim);

function set of int: distinct_values(array[int] of int: x) = {j | j in lb_array(x)..ub_array(x), i in index_set(x) where x[i] = j };

function int: other_axis(int: axis) = if axis == 1 then 2 elseif axis == 2 then 1 else 0 endif;

% Constraints
predicate bl_corners_channeling(int: i) = bl_corners_xs[i] == bl_corners[i, 1] /\ bl_corners_ys[i] == bl_corners[i, 2];
constraint forall(i in ns) (bl_corners_channeling(i));

predicate values_channeling(int: i) = bl_corners_values[i] == coord_to_value(bl_corners_xs[i], bl_corners_ys[i]);
constraint forall(i in ns) (values_channeling(i));

predicate tr_corners_channeling(int: i) = tr_corners[i, 1] == bl_corners_xs[i] + presents_xs[i] /\ tr_corners[i, 2] == bl_corners_ys[i] + presents_ys[i];
constraint forall(i in ns) (tr_corners_channeling(i));

% Reduce presents domains
constraint forall(i in ns) (bl_corners_xs[i] <= w_paper - presents_xs[i] /\ bl_corners_ys[i] <= h_paper - presents_ys[i]);
constraint forall(i in ns) (tr_corners[i, 1] >= presents_xs[i] /\ tr_corners[i, 2] >= presents_ys[i]);

%Presents in the same column ordered by width
%constraint forall(i, j in ns where j > i) ( (bl_corners_xs[i] == bl_corners_xs[j] /\ presents_xs[i] <= presents_xs[j]) -> bl_corners_ys[i] > bl_corners_ys[j]);

%2 presents sum to h_paper or w_paper
%constraint forall(i, j in ns where j > i)(presents_xs[i] == presents_xs[j] /\ presents_ys[i] + presents_ys[j] == h_paper -> bl_corners_ys[j] = presents_ys[i] /\ bl_corners_ys[i] = 0);
%constraint forall(i, j in ns where j > i)(presents_ys[i] == presents_ys[j] /\ presents_xs[i] + presents_xs[j] == w_paper -> bl_corners_xs[j] = presents_xs[i] /\ bl_corners_xs[i] = 0);

%First present in the lower left quadrant
%constraint bl_corners_xs[1] <= w_paper / 2 /\ bl_corners_ys[1] <= h_paper / 2;

%Sum of all the areas of all the placed presents equal to total area
constraint sum(i in ns) ((tr_corners[i, 1] - bl_corners[i, 1]) * (tr_corners[i, 2] - bl_corners[i, 2])) <= total_paper_area;
constraint sum(i in ns) ((tr_corners[i, 1] - bl_corners[i, 1]) * (tr_corners[i, 2] - bl_corners[i, 2])) == total_presents_area;

%All the presents have different bottom left corners
constraint alldifferent(bl_corners_values);

%One present is always placed at the origin
constraint count_eq(bl_corners_values, 0, 1);

%Presents cannot overlap
constraint diffn_k(bl_corners, presents);

constraint cumulative(bl_corners_xs, presents_xs, presents_ys, h_paper);
constraint cumulative(bl_corners_ys, presents_ys, presents_xs, w_paper);


%constraint bl_corners_xs[1] == 0 /\ bl_corners_ys[1] == 0;
%constraint bl_corners_xs[max_present_ind]<=(w_paper - presents_xs[max_present_ind])/2 /\ bl_corners_ys[max_present_ind]<=(h_paper - presents_ys[max_present_ind])/2;

%solve :: int_search(bl_corners_values, dom_w_deg, indomain_min) :: restart_luby(100) satisfy;
%solve minimize sum(bl_corners_values);
solve satisfy;

### Fifth model (with rotation)

In [26]:
%%mzn_model pwp-v5
include "globals.mzn";


%Input values
int: w_paper;
int: h_paper;
int: n;
int: max_dim = max(w_paper, h_paper);
int: total_paper_area = w_paper * h_paper;
int: total_presents_area = sum(areas);
int: min_present_width = min(presents_xs);
int: min_present_height = min(presents_ys);
int: max_present_width = max(presents_xs);
int: max_present_height = max(presents_ys);
int: min_present_dim = min([min_present_width, min_present_height]);
int: max_present_dim = max([max_present_width, max_present_height]);
int: max_present_ind = arg_max(areas);

set of int: ns = 1..n;
set of int: presents_xs_values = distinct_values(presents_xs);
set of int: presents_ys_values = distinct_values(presents_ys);
    
array[ns, 1..2] of int: presents;
array[ns] of int: presents_xs = [presents[i, 1] | i in ns];
array[ns] of int: presents_ys = [presents[i, 2] | i in ns];
array[ns] of int: areas = [presents_xs[i] * presents_ys[i] | i in ns];
array[ns] of int: x_intervals_sizes = [ceil(presents_xs[i] * 0.35) | i in ns]; 
array[1..card(presents_xs_values)] of set of ns: presents_group_xs = [group_presents(i, 1) | i in presents_xs_values];
array[1..card(presents_ys_values)] of set of ns: presents_group_ys = [group_presents(i, 2) | i in presents_ys_values];


% Variables
array[ns, 1..2] of var 0..max_dim - min_present_dim: bl_corners;
array[ns, 1..2] of var min_present_dim..max_dim: tr_corners;
array[ns] of var 0..coord_to_value(w_paper, h_paper): bl_corners_values;
array[ns, 1..4] of var 0..coord_to_value(w_paper, h_paper): corners_values;
array[ns] of var 0..w_paper - min_present_width: bl_corners_xs;
array[ns] of var 0..h_paper - min_present_height: bl_corners_ys;
%array[ns, 1..2] of var 0..w_paper: x_intervals;


% Functions
function var int: coord_to_value(var int: x, var int: y) = x + (y * max_dim);
function int: coord_to_value(int: x, int: y) = x + (y * max_dim);

function set of int: distinct_values(array[int] of int: x) = {j | j in lb_array(x)..ub_array(x), i in index_set(x) where x[i] = j };

function set of ns: group_presents(int: size, int: axis) = {i | i in ns where presents[i, axis] == size};

function int: other_axis(int: axis) = if axis == 1 then 2 elseif axis == 2 then 1 else 0 endif;

predicate is_col_stacking_feasible() = forall(i in 1..card(presents_xs_values))(sum([presents_ys[j] | j in presents_group_xs[i]])>=h_paper);

predicate is_row_stacking_feasible() = forall(i in 1..card(presents_ys_values))(sum([presents_xs[j] | j in presents_group_ys[i]])>=w_paper);


% Constraints
predicate stack_cols(int: i) = let { 
    int: n_presents= card(presents_group_xs[i]);
    array[1..n_presents] of ns: indexes = [j | j in presents_group_xs[i]];
    array[1..n_presents] of 1..h_paper: weights = [presents_ys[j] | j in indexes];
    int: columns = sum(weights) div h_paper;
    array[1..n_presents] of var 1..columns: bins 
    } in bin_packing(h_paper, bins, weights) /\ forall(h,l in 1..n_presents where l>h) ((bins[h]==bins[l]) -> (
        bl_corners_xs[indexes[h]]==bl_corners_xs[indexes[l]] /\ 
        bl_corners_ys[indexes[l]]>=bl_corners_ys[indexes[h]] + presents_ys[indexes[h]]));

predicate stack_rows(int: i) = let { 
    int: n_presents= card(presents_group_ys[i]);
    array[1..n_presents] of ns: indexes = [j | j in presents_group_ys[i]];
    array[1..n_presents] of 1..w_paper: weights = [presents_xs[j] | j in indexes];
    int: columns = sum(weights) div w_paper;
    array[1..n_presents] of var 1..columns: bins 
    } in bin_packing(w_paper, bins, weights) /\ forall(h,l in 1..n_presents where l>h) ((bins[h]==bins[l]) -> (
        bl_corners_ys[indexes[h]]==bl_corners_ys[indexes[l]] /\ 
        bl_corners_xs[indexes[l]]>=bl_corners_xs[indexes[h]] + presents_xs[indexes[h]]));

constraint trace("\nCol_Stacking: "++show(is_col_stacking_feasible())++"\n\n",is_col_stacking_feasible()) -> forall(i in 1..card(presents_xs_values)) (stack_cols(i));
constraint trace("\nRow_Stacking: "++show(is_row_stacking_feasible())++"\n\n",is_row_stacking_feasible() /\ not is_col_stacking_feasible()) -> forall(i in 1..card(presents_ys_values)) (stack_rows(i));

% Channeling constraints
/*
predicate x_intervals_channeling(int: i) = 
    x_intervals[i, 2] == x_intervals[i, 1] + x_intervals_sizes[i] /\
    bl_corners_xs[i] >= x_intervals[i, 1] /\
    bl_corners_xs[i] <= x_intervals[i, 2];
constraint forall(i in ns) (x_intervals_channeling(i));
*/

predicate corners_values_channeling(int: i) = 
    corners_values[i,1] == bl_corners_values[i] /\ 
    corners_values[i,2] == coord_to_value(bl_corners_xs[i] + presents_xs[i], bl_corners_ys[i]) /\
    corners_values[i,3] == coord_to_value(tr_corners[i,1], tr_corners[i,2]) /\
    corners_values[i,4] == coord_to_value(bl_corners_xs[i], bl_corners_ys[i] + presents_ys[i]);
constraint forall(i in ns) (corners_values_channeling(i));

predicate bl_corners_channeling(int: i) = 
    bl_corners_xs[i] == bl_corners[i, 1] /\ 
    bl_corners_ys[i] == bl_corners[i, 2];
constraint forall(i in ns) (bl_corners_channeling(i));

predicate values_channeling(int: i) = 
    bl_corners_values[i] == coord_to_value(bl_corners_xs[i], bl_corners_ys[i]);
constraint forall(i in ns) (values_channeling(i));

predicate tr_corners_channeling(int: i) = 
    tr_corners[i, 1] == bl_corners_xs[i] + presents_xs[i] /\ 
    tr_corners[i, 2] == bl_corners_ys[i] + presents_ys[i];
constraint forall(i in ns) (tr_corners_channeling(i));

% Reduce corners domains based on x intervals
/*
constraint forall(i, j in ns where j != i) (
    not(
        bl_corners_xs[j] < x_intervals[i, 2] + presents_xs[i] - x_intervals_sizes[i] /\ 
        bl_corners_xs[j] > x_intervals[i, 2] /\
        bl_corners_ys[j] >= bl_corners_ys[i] /\
        bl_corners_ys[j] < bl_corners_ys[i] + presents_ys[i]     
    )
);
*/

% Reduce presents domains
constraint forall(i in ns) (bl_corners_xs[i] <= w_paper - presents_xs[i] /\ bl_corners_ys[i] <= h_paper - presents_ys[i]);
constraint forall(i in ns) (tr_corners[i,1] >= presents_xs[i] /\ tr_corners[i,2] >= presents_ys[i]);

% Corners anchor points
predicate anchor_points() =
    let {
        array[1..n * 4] of var int: lin_corners_values = array1d(1..n * 4, corners_values)
    } in
    nvalue(lin_corners_values) <= 2 * n + 2 /\ 
    count_eq(lin_corners_values, coord_to_value(0, 0), 1) /\
    count_eq(lin_corners_values, coord_to_value(0, w_paper), 1) /\
    count_eq(lin_corners_values, coord_to_value(h_paper, 0), 1) /\
    count_eq(lin_corners_values, coord_to_value(w_paper, h_paper), 1);
%constraint anchor_points();

%Presents in the same column ordered by width
%constraint forall(i, j in ns where j > i) ( (bl_corners_xs[i] == bl_corners_xs[j] /\ presents_xs[i] <= presents_xs[j]) -> bl_corners_ys[i] > bl_corners_ys[j]);

%2 presents sum to h_paper or w_paper
%constraint forall(i,j in ns where j > i)(presents_xs[i] == presents_xs[j] /\ presents_ys[i] + presents_ys[j]==h_paper -> bl_corners_ys[j]=presents_ys[i] /\ bl_corners_ys[i] = 0);
%constraint forall(i,j in ns where j > i)(presents_ys[i] == presents_ys[j] /\ presents_xs[i] + presents_xs[j]==w_paper -> bl_corners_xs[j]=presents_xs[i] /\ bl_corners_xs[i] = 0);

%First present in the lower left quadrant
%constraint bl_corners_xs[1] <= w_paper / 2 /\ bl_corners_ys[1] <= h_paper / 2;

%Sum of all the areas of all the placed presents equal to total area
constraint sum(i in ns) ((tr_corners[i, 1] - bl_corners[i, 1]) * (tr_corners[i, 2] - bl_corners[i, 2])) <= total_paper_area;
constraint sum(i in ns) ((tr_corners[i, 1] - bl_corners[i, 1]) * (tr_corners[i, 2] - bl_corners[i, 2])) == total_presents_area;

%All the presents have different bottom left corners
constraint alldifferent(bl_corners_values);

%One present is always placed at the origin
constraint count_eq(bl_corners_values, 0, 1);

%Presents cannot overlap
constraint diffn_k(bl_corners, presents);

constraint cumulative(bl_corners_xs, presents_xs, presents_ys, h_paper);
constraint cumulative(bl_corners_ys, presents_ys, presents_xs, w_paper);


%constraint bl_corners_xs[1] == 0 /\ bl_corners_ys[1] == 0;
%constraint bl_corners_xs[max_present_ind]<=(w_paper - presents_xs[max_present_ind])/2 /\ bl_corners_ys[max_present_ind]<=(h_paper - presents_ys[max_present_ind])/2;

%solve :: int_search(bl_corners_values, dom_w_deg, indomain_min) :: restart_luby(100) satisfy;
%solve minimize sum(bl_corners_values);
solve satisfy;
/*
solve :: seq_search([
             int_search(x_intervals, smallest, indomain_min),
             int_search(bl_corners_xs, smallest, indomain_min),
             int_search(bl_corners_ys, smallest, indomain_min)
         ]) satisfy;
*/

## SMT model

In [ ]:
def z3max(x,y):
    return If(x > y, x, y)

def z3min(x,y):
    return If(x < y, x, y)

bl_corners= [ [ Int("c_%s_%s" % (i, j)) for j in range(2) ] for i in range(n) ]
domain = [And(0<=bl_corners[i][0],bl_corners[i][0]<w_paper,0<=bl_corners[i][1],bl_corners[i][1]< h_paper) for i in range(n)]
overflow = [And(bl_corners[i][0]+presents[i][0]<=w_paper,bl_corners[i][1]+presents[i][1]<=h_paper) for i in range(n)]
pp(bl_corners)
intersection = [ And(
    Or(
        z3max(bl_corners[i][0],bl_corners[j][0])>=z3min(bl_corners[i][0]+presents[i][0],bl_corners[j][0]+presents[j][0]),
        z3max(bl_corners[i][1],bl_corners[j][1])>=z3min(bl_corners[i][1]+presents[i][1],bl_corners[j][1]+presents[j][1])
     )) for i in range(n) for j in range(i+1,n) ]

In [ ]:
%%time
s = Solver()
s.add(domain + overflow + intersection)
s.check()
sol = s.model()
solution=[[sol[bl_corners[i][0]], sol[bl_corners[i][1]]] for i in range(n)]
solution